In [576]:
import pandas as pd 
data =pd.read_csv('0912四个零件.csv',encoding ='UTF-8')

In [577]:
# 因为新零件只有678月真实数据，我们只比较3个月预测值跟真实值的gap

In [578]:
data['日期'] = pd.to_datetime(data['日期'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
new_df = data.groupby(['零件号','日期'])['需求数量'].sum().reset_index()
new_df['日期'] = pd.to_datetime(new_df['日期'])

In [579]:
new_df.head()

,零件号,日期,需求数量
0,2GG821135A,2023-05-03,1
1,2GG821135A,2023-05-28,1
2,2GG821135A,2023-05-30,1
3,2GG821135A,2023-06-10,1
4,2GG821135A,2023-06-11,1


In [580]:
# 生成日期范围
date_range = pd.date_range(start='2023-05-01', end='2023-08-31', freq='D')

# 创建空的DataFrame，准备存储填充后的结果
filled_df = pd.DataFrame()

# 针对每个 SKU 进行填充操作
for sku, group in new_df.groupby('零件号'):
    sku_group = group.set_index('日期').reindex(date_range, fill_value=0).reset_index()
    sku_group['零件号'] = sku
    filled_df = filled_df.append(sku_group, ignore_index=True)

filled_df.head()

/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_86654/3656352837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filled_df = filled_df.append(sku_group, ignore_index=True)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_86654/3656352837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filled_df = filled_df.append(sku_group, ignore_index=True)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_86654/3656352837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filled_df = filled_df.append(sku_group, ignore_index=True)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_86654/3656352837.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas i

,index,零件号,需求数量
0,2023-05-01,2GG821135A,0
1,2023-05-02,2GG821135A,0
2,2023-05-03,2GG821135A,1
3,2023-05-04,2GG821135A,0
4,2023-05-05,2GG821135A,0


In [581]:
data = filled_df.set_index(['index', '零件号'])['需求数量'].unstack()
data.columns.name = None
data = data.reset_index()
new_df = data.copy().rename(columns={'index': 'date'})
new_df['date'] = pd.to_datetime(new_df['date'], format='%Y-%m-%d')
new_df = new_df.set_index('date')
new_df = new_df.asfreq('D')
new_df = new_df.sort_index()
new_df.head()

,2GG821135A,2GG821136A,2GG821168A,5DL945208A
date,,,,
2023-05-01,0,0,0,0
2023-05-02,0,0,0,1
2023-05-03,1,0,0,1
2023-05-04,0,0,0,2
2023-05-05,0,0,0,2


In [582]:
end_train = '2023-06-01'
data_train = new_df.loc[:end_train, :].copy()
data_test  = new_df.loc[end_train:, :].copy()

print(
    f"Train dates : {data_train.index.min()} --- {data_train.index.max()}   "
    f"(n={len(data_train)})"
)
print(
    f"Test dates  : {data_test.index.min()} --- {data_test.index.max()}   "
    f"(n={len(data_test)})"
)



import numpy as np
import pandas as pd
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.linear_model import Ridge

sku_list = data_train.columns

# 每批次处理的SKU数量
batch_size = 200

# 创建空的字典，用于存储每个时间序列的预测模型
forecasters = {}

# 循环遍历每批次SKU，分别训练预测模型
for i in range(0, len(sku_list), batch_size):
    batch_skus = sku_list[i:i+batch_size]
    
    for sku in batch_skus:
        forecaster = ForecasterAutoreg(
                         regressor          = Ridge(random_state=123),
                         lags               = 4,
                     )
        
        # 拟合模型
        forecaster.fit(y=data_train[sku])
        
        forecasters[sku] = forecaster
    
    print(f"Finished training batch {i//batch_size + 1}/{len(sku_list)//batch_size + 1}")

# 进行未来预测
predictions = pd.DataFrame()

for sku, forecaster in forecasters.items():
    forecast = forecaster.predict(steps=90)
    predictions[sku] = forecast

# 打印前 3 个预测结果
predictions.head(3)

Train dates : 2023-05-01 00:00:00 --- 2023-06-01 00:00:00   (n=32)
Test dates  : 2023-06-01 00:00:00 --- 2023-08-31 00:00:00   (n=92)
Finished training batch 1/1


,2GG821135A,2GG821136A,2GG821168A,5DL945208A
2023-06-02,0.029412,0.330389,0.162921,1.121965
2023-06-03,-0.059689,0.358061,0.174791,1.029936
2023-06-04,0.067499,0.306930,0.145201,0.988952


In [583]:
predictions[predictions < 0.1] = 0
data_full =data_train.append(predictions)

/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_86654/3055008364.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_full =data_train.append(predictions)


In [584]:
test_pred = data_full.reset_index()
test_pred["year"] = pd.to_datetime(test_pred['index']).dt.year.astype(int)
test_pred["month"] = pd.to_datetime(test_pred['index']).dt.month.astype(int)
pred_info = test_pred.groupby(['year','month']).sum().reset_index()



pred_info=pred_info.set_index(['year','month']).stack()
pred_info=pred_info.rename_axis(index=['year','month','零件号'])
pred_info.name='pred_values'
pred_info = pred_info.reset_index()
pred_info.head()

,year,month,零件号,pred_values
0,2023,5,2GG821135A,3.0
1,2023,5,2GG821136A,7.0
2,2023,5,2GG821168A,5.0
3,2023,5,5DL945208A,31.0
4,2023,6,2GG821135A,0.0


In [585]:
data_test_real = data.reset_index().drop('level_0',axis =1)
data_test_real["year"] = pd.to_datetime(data_test_real['index']).dt.year.astype(int)
data_test_real["month"] = pd.to_datetime(data_test_real['index']).dt.month.astype(int)
real_info = data_test_real.groupby(['year','month']).sum().reset_index()
real_info=real_info.set_index(['year','month']).stack()
real_info=real_info.rename_axis(index=['year','month','零件号'])
real_info.name='real_values'
real_info = real_info.reset_index()
real_info.head()

,year,month,零件号,real_values
0,2023,5,2GG821135A,3
1,2023,5,2GG821136A,7
2,2023,5,2GG821168A,5
3,2023,5,5DL945208A,31
4,2023,6,2GG821135A,6


In [586]:
output = pd.merge(real_info,pred_info,on =['year','month','零件号'],how ='inner')
output = output[(output['year']==2023)&(output['month'].isin([6,7,8]))]

for i in ['real_values','pred_values']:
    output[i]= [1 if x ==0 else x for x in output[i]]
output['mape'] = (abs(output['real_values'] - output['pred_values']) 
                                 / output['real_values']) 
output.tail(20)

,year,month,零件号,real_values,pred_values,mape
4,2023,6,2GG821135A,6,1.000000,0.833333
5,2023,6,2GG821136A,14,7.460779,0.467087
6,2023,6,2GG821168A,4,5.125386,0.281347
7,2023,6,5DL945208A,30,29.991343,0.000289
8,2023,7,2GG821135A,17,1.000000,0.941176
9,2023,7,2GG821136A,20,7.750002,0.612500
10,2023,7,2GG821168A,10,5.535715,0.446429
11,2023,7,5DL945208A,37,30.836702,0.166576
12,2023,8,2GG821135A,31,1.000000,0.967742
13,2023,8,2GG821136A,37,7.500000,0.797297


In [587]:
output.mape.mean()

0.5227286636612386

In [588]:
import pandas as pd 
data = pd.read_csv('0912四个零件.csv',encoding='utf-8')
data['日期'] = pd.to_datetime(data['日期'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
data = data[(data['日期']>='2023-06-01')&(data['日期']<'2023-09-01')]
data["month"] = pd.to_datetime(data['日期']).dt.month.astype(int)
sku_frequency = data.groupby(['零件号','month']).size().reset_index(name='sku_freq')
abc_info = data.drop_duplicates('零件号')[['零件号','需求频次abc分类']]

In [589]:
my_pred_info =output[['零件号','month','pred_values','real_values']]
my_pred_info.columns = ['零件号','month','algo_预测值','real_values']


svg_pred_info =pd.read_csv('四个零件预测.csv',encoding = 'utf-8')
svg_pred_info['日期'] = pd.to_datetime(svg_pred_info['日期'], format='%Y%m').dt.strftime('%Y-%m-%d')
svg_pred_info = svg_pred_info[(svg_pred_info['日期']>='2023-06-01')&(svg_pred_info['日期']<'2023-09-01')]
svg_pred_info["month"] = pd.to_datetime(svg_pred_info['日期']).dt.month.astype(int)
svg_pred_info = svg_pred_info[['零件代码','month','预测值']]
svg_pred_info.columns = ['零件号','month','svg_预测值']


poc_pred_info = pd.merge(my_pred_info,svg_pred_info,on = ['零件号','month'],how ='inner')
poc_pred = pd.merge(poc_pred_info,abc_info,on = ['零件号'],how ='inner')
# poc_pred = pd.merge(poc_pred,sku_frequency,on = ['零件号','month'],how ='inner')


poc_pred.head()



,零件号,month,algo_预测值,real_values,svg_预测值,需求频次abc分类
0,2GG821135A,6,1.000000,6,3.000000,C
1,2GG821135A,7,1.000000,17,5.000000,C
2,2GG821135A,8,1.000000,31,11.000000,C
3,2GG821136A,6,7.460779,14,7.000000,C
4,2GG821136A,7,7.750002,20,11.666667,C


In [590]:
poc_pred['algo_mape'] = (abs(poc_pred['real_values'] - poc_pred['algo_预测值']) 
                                 / poc_pred['real_values']) 
poc_pred['svg_mape'] = (abs(poc_pred['real_values'] - poc_pred['svg_预测值']) 
                                 / poc_pred['real_values']) 

In [591]:
poc_pred

,零件号,month,algo_预测值,real_values,svg_预测值,需求频次abc分类,algo_mape,svg_mape
0,2GG821135A,6,1.000000,6,3.000000,C,0.833333,0.500000
1,2GG821135A,7,1.000000,17,5.000000,C,0.941176,0.705882
2,2GG821135A,8,1.000000,31,11.000000,C,0.967742,0.645161
3,2GG821136A,6,7.460779,14,7.000000,C,0.467087,0.500000
4,2GG821136A,7,7.750002,20,11.666667,C,0.612500,0.416667
5,2GG821136A,8,7.500000,37,15.833333,C,0.797297,0.572072
6,2GG821168A,6,5.125386,4,5.000000,D,0.281347,0.250000
7,2GG821168A,7,5.535715,10,4.333333,D,0.446429,0.566667
8,2GG821168A,8,5.357143,13,7.166667,D,0.587912,0.448718
9,5DL945208A,6,29.991343,30,35.754012,B,0.000289,0.191800


In [592]:
poc_pred.algo_mape.mean()

0.5227286636612386

In [593]:
poc_pred.svg_mape.mean()

0.4029295726580196

In [594]:
poc_pred.to_excel('four_sku_pred_info_20230913.xlsx')